# Upload Data to SEED Instance

In [6]:
# import libraries
import os
import json 
import copy

from datetime import date
from pathlib import Path

from pyseed.seed_client import SeedClient

curdir = Path(os.curdir).absolute().parent
seed_config = curdir / Path('seed-config-local.json')
org_name = 'Program Tracking 1'
# seed_config = curdir / Path('seed-config-dev1.json')
# org_name = 'Program Tracking 1'
# seed_config = curdir / Path('seed-config-prod.json')
# org_name = 'Program Tracking 1'
data_dir = curdir / 'data'

client = SeedClient(
    None,
    connection_config_filepath=seed_config,
)

In [8]:
client.get_org_by_name(org_name, set_org_id=True)

client_info = client.instance_information()
print("You are connecting to the following instance:")
print(f"\tHost: {client_info['host']}")
print(f"\tVersion: {client_info['version']}")
print(f"\tSHA: {client_info['sha']}")
print(f"\tUsername: {client_info['username']}")

You are connecting to the following instance:
	Host: http://127.0.0.1:8000/
	Version: 2.18.1
	SHA: 6bec6092b
	Username: nicholas.long@nrel.gov


In [29]:
# The data_dir is in SEED's repo
data_dir = Path().absolute().parent.parent / 'seed' / 'seed' / 'tests' / 'data' / 'bps'

# path to where the mapping files exist
mappings_dir = Path().parent.absolute() / 'data' / 'bps'

if not data_dir.exists():
    print(f"Data directory '{data_dir}' does not exist. Please clone the SEED repo and try again.")
    
if not mappings_dir.exists():
    print(f"Mapping directory '{mappings_dir}' does not exist, which should be in data/bps")
    
print(f"SEED data will be loaded from {data_dir}")
print(f"SEED mappings will be loaded from {mappings_dir}")

SEED data will be loaded from /Users/nlong/working/seed/seed/seed/tests/data/bps
SEED mappings will be loaded from /Users/nlong/working/seed/pyseed-examples/examples/data/bps


In [30]:
# Import data from downloaded ESPM 

# Creating mapping profile
client.create_or_update_column_mapping_profile_from_file('ESPM Webservice', mappings_dir / 'espm-webservice-mappings.csv')


{'id': 31,
 'profile_type': 'Normal',
 'name': 'ESPM Webservice',
 'mappings': [{'from_field': 'address_1',
   'from_units': '',
   'to_table_name': 'PropertyState',
   'to_field': 'address_line_1'},
  {'from_field': 'address_2',
   'from_units': '',
   'to_table_name': 'PropertyState',
   'to_field': 'address_line_2'},
  {'from_field': 'city',
   'from_units': '',
   'to_table_name': 'PropertyState',
   'to_field': 'city'},
  {'from_field': 'custom_property_id_1_id',
   'from_units': '',
   'to_table_name': 'PropertyState',
   'to_field': 'audit_template_building_id'},
  {'from_field': 'custom_property_id_1_name',
   'from_units': '',
   'to_table_name': 'PropertyState',
   'to_field': 'Audit Template ID'},
  {'from_field': 'energy_star_score',
   'from_units': '',
   'to_table_name': 'PropertyState',
   'to_field': 'energy_score'},
  {'from_field': 'national_median_site_eui',
   'from_units': '',
   'to_table_name': 'PropertyState',
   'to_field': 'National Median Site EUI'},
  {'fro

In [33]:
for cycle_year in range(2019, 2023):
    cycle = client.get_or_create_cycle(f"{str(cycle_year)}", date(cycle_year, 1, 1), date(cycle_year, 12, 31), set_cycle_id=True)

    # upload CBL data
    upload_file_name = f'CBL-building-performance-standards-sample-{cycle_year}.xlsx'
    if (data_dir / upload_file_name).exists():
        print(f'uploading {upload_file_name}')
        client.upload_and_match_datafile(
            'cbl-data',
            str(data_dir / upload_file_name),
            'cbl mappings',
            str(mappings_dir / 'cbl-mappings.csv'),
            import_meters_if_exist=False,
        )   
    else:
        print(f'Could not find {data_dir / upload_file_name}, exiting...')
        exit()

  

uploading CBL-building-performance-standards-sample-2019.xlsx
uploading CBL-building-performance-standards-sample-2020.xlsx
uploading CBL-building-performance-standards-sample-2021.xlsx
uploading CBL-building-performance-standards-sample-2022.xlsx


In [34]:
# upload targets
for cycle_year in range(2019, 2023):
    cycle = client.get_or_create_cycle(f"{str(cycle_year)}", date(cycle_year, 1, 1), date(cycle_year, 12, 31), set_cycle_id=True)

    upload_file_name = f'BPS-sample-Targets-{cycle_year}.xlsx'
    if (data_dir / upload_file_name).exists():
        print(f'uploading {upload_file_name}')
        client.upload_and_match_datafile(
            'bps-target-data',
            str(data_dir / upload_file_name),
            'bps target mappings',
            str(mappings_dir / 'bps-targets-mappings.csv'),
            import_meters_if_exist=False,
        )
    else:
        print(f'Could not find {data_dir / upload_file_name}, exiting...')

uploading BPS-sample-Targets-2019.xlsx
uploading BPS-sample-Targets-2020.xlsx
uploading BPS-sample-Targets-2021.xlsx
uploading BPS-sample-Targets-2022.xlsx
